In [2]:
# Исследовать структуру html-файлов, чтобы произвести парсинг всех данных. 
# В каждом файле содержится информация об одном объекте из случайной предметной области. 
# Полученные данные собрать и записать в json. 
# Выполните также ряд операций с данными:
# 1) отсортируйте значения по одному из доступных полей
# 2) выполните фильтрацию по другому полю (запишите результат отдельно)
# 3) для одного выбранного числового поля посчитайте статистические характеристики (сумма, мин/макс, среднее и т.д.)
# 4) для одного текстового поля посчитайте частоту меток

In [3]:
from bs4 import BeautifulSoup
import re
import json
import math
import collections
import pandas as pd

In [4]:
def handle_file(file_name):
    with open(file_name, encoding = 'utf-8') as file:
        text = ""
        for row in file.readlines():
            text += row
        item = dict()
        site = BeautifulSoup(text, 'html.parser')
        # print(site.prettify())
        item['category'] = site.find_all("span", string = re.compile('Категория:'))[0].get_text().split(':')[1].strip()
        item['title'] = site.find_all('h1')[0].get_text().strip()
        item['author'] = site.find_all("p", attrs = {'class' : 'author-p'})[0].get_text().strip()
        item['pages'] = int(site.find_all("span", attrs = {'class' : 'pages'})[0].get_text().split(':')[1].replace(" страниц", "").strip())
        item['year'] = int(site.find_all('span', attrs = {'class': 'year'})[0].get_text().replace("Издано в ", "").strip())
        item['ISBN'] = site.find_all('span', string = re.compile('ISBN'))[0].get_text(). split(':')[1].strip()
        item['description'] = site.find_all("p")[1].get_text().replace("Описание \n", "").strip()
        item['img'] = site.find_all('img')[0]['src']
        item['rate'] = float(site.find_all('span', string = re.compile('Рейтинг:'))[0].get_text().split(':')[1].strip())
        item['views'] = int(site.find_all('span', string = re.compile('Просмотры:'))[0].get_text().split(':')[1].strip())
        return item

In [5]:
items = []
for i in range(1,1000):
  file_name = f'./1/{i}.html'
  result = handle_file(file_name)
  items.append(result)
print(items[1:5])

[{'category': 'фэнтези', 'title': 'Три товарища', 'author': 'Ясунари Кавабата', 'pages': 446, 'year': 2006, 'ISBN': '301-2019317665', 'description': 'Описание\n     write it yourself', 'img': 'https://upload.buybook.org/387964.webp', 'rate': 3.7, 'views': 50110}, {'category': 'любовный роман', 'title': 'О дивный новый мир', 'author': 'Юстейн Гордер', 'pages': 293, 'year': 1840, 'ISBN': '127-2255174171', 'description': 'Описание\n     very impressive', 'img': 'https://upload.abc-book.it/491771.webp', 'rate': 2.4, 'views': 75183}, {'category': 'фэнтези', 'title': 'Молот ведьм', 'author': 'Уильям Голдинг', 'pages': 392, 'year': 1971, 'ISBN': '401-4364312517', 'description': 'Описаниеwrite it yourself', 'img': 'https://upload.abc-book.it/747752.svg', 'rate': 0.4, 'views': 18092}, {'category': 'приключения', 'title': 'Лолита', 'author': 'Сигрид Унсет', 'pages': 413, 'year': 2005, 'ISBN': '249-5825832197', 'description': 'Описание\n     very impressive', 'img': 'https://upload.abc-book.it/32

In [6]:
with open('result_all_17_1.json', 'w', encoding = 'utf-8') as f:
    f.write(json.dumps(items, ensure_ascii=False))

In [7]:
# Отсортируeм значения по полю ['views'] в убывающем порядке:
items = sorted(items, key = lambda x: x['views'], reverse = True)
print(items[1:5])

[{'category': 'биография', 'title': 'Исчезнувшая', 'author': 'Казуо Исигуро', 'pages': 250, 'year': 1953, 'ISBN': '982-1556130853', 'description': 'Описаниеvery impressive', 'img': 'https://upload.buybook.org/811630.webp', 'rate': 2.4, 'views': 99913}, {'category': 'любовный роман', 'title': 'Приключения Гекльберри Финна', 'author': 'Иван Тургенев', 'pages': 303, 'year': 1983, 'ISBN': '840-1695683117', 'description': 'Описаниеvery impressive', 'img': 'https://upload.booook.ru/630257.png', 'rate': 4.2, 'views': 99799}, {'category': 'фэнтези', 'title': 'Сквозь время', 'author': 'Уильям Голдинг', 'pages': 349, 'year': 1818, 'ISBN': '155-1248159016', 'description': 'Описание\n     small', 'img': 'https://upload.booook.ru/895447.jpg', 'rate': 2.6, 'views': 99636}, {'category': 'исторический роман', 'title': 'Плавучий трава', 'author': 'Фрэнсис Скотт Фицджеральд', 'pages': 376, 'year': 1817, 'ISBN': '585-9386102435', 'description': 'Описаниеmiddle', 'img': 'https://upload.buybook.org/388241.

In [8]:
# Выполним фильтрацию по полю ['category'] = 'любовный роман':
filtered_items = []
for book in items:
    if book['category'] == 'любовный роман':
        filtered_items.append(book)
print(filtered_items[1:5])        

[{'category': 'любовный роман', 'title': 'Ночная стража', 'author': 'Антон Чехов', 'pages': 272, 'year': 1933, 'ISBN': '880-6149219939', 'description': 'Описаниеwrite it yourself', 'img': 'https://upload.booook.ru/18509.jpeg', 'rate': 1.4, 'views': 93723}, {'category': 'любовный роман', 'title': 'Приключения Гекльберри Финна', 'author': 'Иван Тургенев', 'pages': 181, 'year': 1946, 'ISBN': '846-6146595386', 'description': 'Описание\n     small', 'img': 'https://upload.book.com/118816.webp', 'rate': 4.6, 'views': 93531}, {'category': 'любовный роман', 'title': 'Три товарища', 'author': 'Дон Делилло', 'pages': 318, 'year': 1836, 'ISBN': '801-9089266513', 'description': 'Описаниеmiddle', 'img': 'https://upload.freebook.xyz/226929.svg', 'rate': 3.9, 'views': 93422}, {'category': 'любовный роман', 'title': 'Возвращение в Брайдсхед', 'author': 'Федор Достоевский', 'pages': 230, 'year': 1888, 'ISBN': '148-2943467438', 'description': 'Описаниеmiddle', 'img': 'https://upload.abc-book.it/588996.j

In [9]:
with open("result_filtered_17_1.json", "w", encoding = 'utf-8') as file:
    file.write(json.dumps(filtered_items, ensure_ascii=False))

In [10]:
# для числового поля ['year'] посчитаем статистические характеристики (сумма, мин/макс, среднее и т.д.)
stats_items = []
df = pd.DataFrame(items)
pd.set_option('display.float_format', '{:.1f}'.format)
stats = df['year'].agg(['max', 'min', 'mean', 'median', 'std']).to_dict()
stats_items.append(stats)
print(stats_items)

[{'max': 2023.0, 'min': 1800.0, 'mean': 1911.964964964965, 'median': 1912.0, 'std': 66.44766631335337}]


In [12]:
# для текстового поля ['author'] посчитайте частоту меток
text_items = []
text = [item['author'] for item in items]
f1 = collections.Counter(text)
text_items.append(f1)
print(text_items)

[Counter({'Габриэль Гарсиа Маркес': 32, 'Джордж Оруэлл': 28, 'Стивен Кинг': 27, 'Владимир Набоков': 26, 'Алексей Толстой': 26, 'Иэн Бэнкс': 26, 'Генрих Сенкевич': 25, 'Патрик Зюскинд': 25, 'Эрнест Хемингуэй': 25, 'Николай Гоголь': 24, 'Артур Конан Дойл': 24, 'Иван Тургенев': 23, 'Тед Хьюз': 23, 'Федор Достоевский': 22, 'Марк Твен': 22, 'Эрих Мария Ремарк': 22, 'Антон Чехов': 22, 'Харпер Ли': 22, 'Лев Толстой': 22, 'Джозеф Конрад': 22, 'Колин Маккалоу': 22, 'Уильям Голдинг': 21, 'Роберт Льюис Стивенсон': 21, 'Синклер Льюис': 21, 'Алан Александр Милн': 21, 'Клайв Стейплз Льюис': 20, 'Борис Пастернак': 20, 'Дж. Р. Р. Толкин': 20, 'Станислав Лем': 20, 'Сигрид Унсет': 19, 'Сельма Лагерлёф': 19, 'Фрэнсис Скотт Фицджеральд': 18, 'Исаак Бабель': 18, 'Жюль Верн': 18, 'Ярослав Гашек': 18, 'Майн Рид': 17, 'Харуки Мураками': 17, 'Дон Делилло': 17, 'Герберт Уэллс': 16, 'Джек Лондон': 16, 'Казуо Исигуро': 15, 'Джейн Остин': 15, 'Оскар Уайльд': 15, 'Салман Рушди': 14, 'Ясунари Кавабата': 14, 'Алексан